In [20]:
### Before we submit, review which modules we actually use !!!

%load_ext autoreload
%autoreload 2

import warnings
import calendar

import pandas as pd
import numpy as np
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.pylab as pltlab

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso, LinearRegression, ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split

import pipeline as pipe
pd.set_option('display.max_columns', None)

warnings.filterwarnings('ignore')
%matplotlib inline
sns.set(rc={'figure.figsize':(11, 4)})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data Import & Exploration

In [2]:
data = pd.read_pickle('data/final_dataset.pk1')

In [3]:
pipe.find_outliers(data, 'median_income', 0, 10000000)

Number of outliers found: 0
Outlier values found: []


,name,total_pop,median_income,state,county,county_name,state_name,prop_white,prop_black,prop_hisp,log_med_income,prop_no_internet,prop_ba,prop_services,pop_density,FIPS,covid_cases,Testing_Rate,gov_party,election_diff,Apr-19,Mar-19,Feb-20,Mar-20,days_closed,yearly_change,monthly_change


In [4]:
data.head()

,name,total_pop,median_income,state,county,county_name,state_name,prop_white,prop_black,prop_hisp,log_med_income,prop_no_internet,prop_ba,prop_services,pop_density,FIPS,covid_cases,Testing_Rate,gov_party,election_diff,Apr-19,Mar-19,Feb-20,Mar-20,days_closed,yearly_change,monthly_change
0,"Washington County, Mississippi",47086,30834.0,28,151,Washington,Mississippi,0.256913,0.721701,0.015482,10.336373,0.336958,0.187937,0.100274,62.178441,28151.0,77,2291.374085,1,-0.364695,6.4,7.5,7.4,7.0,17.0,-0.066667,-0.054054
1,"Perry County, Mississippi",12028,39007.0,28,111,Perry,Mississippi,0.787745,0.196874,0.015048,10.571496,0.310103,0.109539,0.078850,18.434583,28111.0,27,2291.374085,1,0.536635,5.7,6.2,6.8,6.3,17.0,0.016129,-0.073529
2,"Choctaw County, Mississippi",8321,37203.0,28,019,Choctaw,Mississippi,0.676722,0.311982,0.003966,10.524145,0.368837,0.176582,0.014207,19.794546,28019.0,13,2291.374085,1,0.386224,4.8,5.3,5.4,5.0,17.0,-0.056604,-0.074074
3,"Itawamba County, Mississippi",23480,40510.0,28,057,Itawamba,Mississippi,0.909114,0.071593,0.015332,10.609304,0.226281,0.134468,0.057524,44.138916,28057.0,57,2291.374085,1,0.755161,4.1,4.4,4.7,4.7,17.0,0.068182,0.000000
4,"Carroll County, Mississippi",10129,43060.0,28,015,Carroll,Mississippi,0.643992,0.345839,0.002863,10.670350,0.332969,0.145006,0.057415,15.774563,28015.0,38,2291.374085,1,0.383321,5.5,5.8,6.2,6.0,17.0,0.034483,-0.032258


In [5]:
pipe.get_summary_stats(data)

total_pop  median_income   prop_white   prop_black    prop_hisp  \
count  2.826000e+03    2825.000000  2826.000000  2826.000000  2826.000000   
mean   1.117590e+05   51714.241416     0.827329     0.098748     0.091962   
std    3.414925e+05   13821.224638     0.164131     0.149708     0.135249   
min    4.180000e+02   20188.000000     0.093534     0.000000     0.000000   
25%    1.412200e+04   42491.000000     0.757614     0.008110     0.021500   
50%    3.013550e+04   49936.000000     0.889050     0.028182     0.041568   
75%    7.725500e+04   57886.000000     0.947572     0.117748     0.095686   
max    1.009805e+07  136268.000000     0.997743     0.874123     0.990688   

       log_med_income  prop_no_internet      prop_ba  prop_services  \
count     2825.000000       2826.000000  2826.000000    2825.000000   
mean        10.821023          0.225514     0.217787       0.083571   
std          0.251809          0.086132     0.096556       0.034479   
min          9.912844          0

After reviewing the summary statistics, we confirmed that the median income is non-negative. We also looked into the potential outlier on population density and confirmed that that value is New York City.

In [6]:
data.dtypes

name                 object
total_pop             int64
median_income       float64
state                object
county               object
county_name          object
state_name           object
prop_white          float64
prop_black          float64
prop_hisp           float64
log_med_income      float64
prop_no_internet    float64
prop_ba             float64
prop_services       float64
pop_density         float64
FIPS                float64
covid_cases           int64
Testing_Rate        float64
gov_party             int64
election_diff       float64
Apr-19              float64
Mar-19              float64
Feb-20              float64
Mar-20              float64
days_closed         float64
yearly_change       float64
monthly_change      float64
dtype: object

# Model Fitting & Evaluation

In [47]:
best_models = {}

## Monthly Data

### With States

In [48]:
# Select variables from full dataframe
df_mo_st = data.drop(['Apr-19','Mar-20','Feb-20','Mar-20','median_income',
                  'yearly_change','FIPS','name','state_name','county_name','county'], axis=1)

In [49]:
# One-hot encode state
df_mo_st = pipe.hot_encode(df_mo_st, ['state'])

# Create training and testing sets
train, test = pipe.split_data(df_mo_st)

# Impute/nomralize continuous variables
numeric_cols = train.select_dtypes(include=['float64']).columns
train, test = pipe.impute_missing(train, test, numeric_cols)
train, test = pipe.normalize(train, test, numeric_cols)

# Separate features and targets
train_features = train.drop('monthly_change', axis=1)
test_features = test.drop('monthly_change', axis=1)

train_target = train['monthly_change']
test_target = test['monthly_change']

Training set contains 2260 observations
Testing set contains 566 observation

Imputing log_med_income missing values with median 10.819318178577827
Imputing prop_services missing values with median 0.0790869921304704


### Ridge

In [50]:
# Set up Pipeline for Ridge
pipeline = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)),
    ('ridge', Ridge(alpha=0.1))
])

alpha_range = np.arange(0,1,0.1)
params = {
          'ridge__alpha': alpha_range,
          'poly__degree': (1,2)
         }

grid_model, grid_model_result, cv_results = pipe.run_gridsearch(pipeline, params, train_features, train_target, verbose=1)

cv_results

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   16.1s finished


,param_poly__degree,rank_test_score,mean_test_score
0,1,1,-0.169051
1,2,2,-4.128366


In [51]:
print(grid_model_result.best_estimator_, '\n')

# Evaluate model
mse, r2, adj = pipe.evaluate_model(grid_model, test_features, test_target)

d = {'model': grid_model_result,
     'mse': mse,
     'r2': r2}

best_models['state_ridge'] = d

# Rate coefficients
pipe.rank_coefs(grid_model_result, train_features.columns)

Pipeline(memory=None,
         steps=[('poly',
                 PolynomialFeatures(degree=1, include_bias=False,
                                    interaction_only=False, order='C')),
                ('ridge',
                 Ridge(alpha=0.1, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False) 

Mean Squared Error: 0.17870094572336545
R-Squared: 0.8251238972397802
Adjusted R-Squared: 0.801993991864681


,feature,coefficient,coefficient (absolute)
20,state_08,2.70612,2.706121e+00
43,state_32,2.45017,2.450172e+00
26,state_15,-1.736,1.735998e+00
33,state_22,1.68121,1.681206e+00
42,state_31,1.56017,1.560169e+00
...,...,...,...
12,election_diff,-0.00791619,7.916192e-03
8,pop_density,-0.00120876,1.208756e-03
9,covid_cases,3.3044e-06,3.304398e-06
0,total_pop,1.66046e-07,1.660456e-07


In [ ]:
# Lasso
pipeline = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)),
    ('lasso', Lasso(alpha=0.1))
])

params = {
          'lasso__alpha': alpha_range,
          'poly__degree': (1,2)
         }

grid_model, grid_model_result, cv_results = pipe.run_gridsearch(pipeline, params, train_features, train_target, verbose=1)

cv_results

In [19]:
print(grid_model_result.best_estimator_, '\n')

# Evaluate model
mse, r2, adj = pipe.evaluate_model(grid_model, test_features, test_target)

d = {'model': grid_model_result,
     'mse': mse,
     'r2': r2}

best_models['state_lasso'] = d

# Rank coefficients
pipe.rank_coefs(grid_model_result, train_features.columns)

Pipeline(steps=[('poly', PolynomialFeatures(degree=1, include_bias=False)),
                ('ridge', Ridge(alpha=0.9))]) 

Mean Squared Error: 0.17901481774533548
R-Squared: 0.824816743207968
Adjusted R-Squared: 0.8016462122495029


,feature,coefficient,coefficient (absolute)
20,state_08,0.362565,3.625650e-01
33,state_22,0.247366,2.473659e-01
58,state_48,0.246354,2.463540e-01
64,state_55,-0.229637,2.296374e-01
42,state_31,0.211224,2.112235e-01
...,...,...,...
8,pop_density,-0.00152606,1.526061e-03
11,gov_party,-0.000735508,7.355081e-04
9,covid_cases,3.40473e-06,3.404729e-06
0,total_pop,1.65259e-07,1.652587e-07


### Linear Regression

In [42]:
pipeline_l = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)), ('linreg', LinearRegression())
])

params_l = {
          'poly__degree': (1,2)
         }

grid_model, grid_model_result, cv_results = pipe.run_gridsearch(pipeline, params, train_features, train_target, verbose=1)

cv_results

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   25.5s finished


,param_poly__degree,rank_test_score,mean_test_score
0,1,1,-0.942384
1,2,2,-2.866127


In [43]:
print(grid_model_result.best_estimator_, '\n')

# Evaluate model
mse, r2, adj = pipe.evaluate_model(grid_model, test_features, test_target)

d = {'model': grid_model_result,
     'mse': mse,
     'r2': r2}

best_models['state_linear'] = d

# Rank coefficients
pipe.rank_coefs(grid_model_result, train_features.columns)

Pipeline(memory=None,
         steps=[('poly',
                 PolynomialFeatures(degree=1, include_bias=False,
                                    interaction_only=False, order='C')),
                ('lasso',
                 Lasso(alpha=0.1, copy_X=True, fit_intercept=True,
                       max_iter=1000, normalize=False, positive=False,
                       precompute=False, random_state=None, selection='cyclic',
                       tol=0.0001, warm_start=False))],
         verbose=False) 

Mean Squared Error: 0.86586640646064
R-Squared: 0.07314675610482002
Adjusted R-Squared: -0.049443051704963326


,feature,coefficient,coefficient (absolute)
3,prop_hisp,0.128849,0.128849
13,Mar-19,-0.046523,0.046523
5,prop_no_internet,0.0344106,0.034411
14,days_closed,-0.00756243,0.007562
9,covid_cases,-7.03838e-06,0.000007
...,...,...,...
30,state_19,0,0.000000
31,state_20,-0,0.000000
32,state_21,0,0.000000
1,prop_white,-0,0.000000


### Elastic Net

In [37]:
pipeline_l = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)), ('en',ElasticNet())
])

params_l = {'en__alpha': alpha_range,
          'poly__degree': (1,2)
         }

grid_model, grid_model_result, cv_results = pipe.run_gridsearch(pipeline, params, train_features, train_target, verbose=1)

cv_results

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

In [ ]:
print(grid_model_result.best_estimator_, '\n')

# Evaluate model
mse, r2, adj = pipe.evaluate_model(grid_model, test_features, test_target)

d = {'model': grid_model_result,
     'mse': mse,
     'r2': r2}

best_models['state_en'] = d

# Rank coefficients
pipe.rank_coefs(grid_model_result, train_features.columns)

### Without States

In [20]:
# Select variables from full dataframe
df_mo = data.drop(['Apr-19','Mar-20','Feb-20','Mar-20','median_income',
                  'yearly_change','FIPS','name','state_name','county_name','county','state'], axis=1)

In [21]:
# Create training and testing sets
train, test = pipe.split_data(df_mo)

# Impute/normalize continuous variables
numeric_cols = train.select_dtypes(include=['float64']).columns #Normalizes state one-hots as well
train, test = pipe.impute_missing(train, test, numeric_cols)
train, test = pipe.normalize(train, test, numeric_cols)

# Separate features and targets                              
train_features = train.drop('monthly_change', axis=1)
test_features = test.drop('monthly_change', axis=1)

train_target = train['monthly_change']
test_target = test['monthly_change']

Training set contains 2260 observations
Testing set contains 566 observation

Imputing log_med_income missing values with median 10.819318178577827
Imputing prop_services missing values with median 0.0790869921304704


### Ridge

In [ ]:
# Set up Pipeline for Ridge
pipeline = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)),
    ('ridge', Ridge(alpha=0.1))
])

alpha_range = np.arange(0,1,0.1)
params = {
          'ridge__alpha': alpha_range,
          'poly__degree': (1,2)
         }

grid_model, grid_model_result, cv_results = pipe.run_gridsearch(pipeline, params, train_features, train_target, verbose=1)

cv_results

In [23]:
print(grid_model_result.best_estimator_, '\n')

# Evaluate model
mse, r2, adj = pipe.evaluate_model(grid_model, test_features, test_target)

d = {'model': grid_model_result,
     'mse': mse,
     'r2': r2}

best_models['ridge'] = d

# Rank coefficients
pipe.rank_coefs(grid_model_result, train_features.columns)

Pipeline(steps=[('poly', PolynomialFeatures(degree=1, include_bias=False)),
                ('ridge', Ridge(alpha=0.9))]) 

Mean Squared Error: 0.7667409541331213
R-Squared: 0.24967005998379077
Adjusted R-Squared: 0.22920651616516685


,feature,coefficient,coefficient (absolute)
2,prop_black,0.406446,4.064456e-01
3,prop_hisp,0.332707,3.327068e-01
12,election_diff,0.267847,2.678465e-01
1,prop_white,0.218845,2.188450e-01
11,gov_party,-0.203038,2.030378e-01
6,prop_ba,0.172722,1.727218e-01
13,Mar-19,-0.154737,1.547369e-01
7,prop_services,0.116795,1.167948e-01
4,log_med_income,-0.0960348,9.603484e-02
10,Testing_Rate,-0.0945844,9.458437e-02


### Lasso

In [ ]:
# Lasso
pipeline = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)),
    ('lasso', Lasso(alpha=0.1))
])

params = {
          'lasso__alpha': alpha_range,
          'poly__degree': (1,2)
         }

grid_model, grid_model_result, cv_results = pipe.run_gridsearch(pipeline, params, train_features, train_target, verbose=1)

cv_results

In [ ]:
print(grid_model_result.best_estimator_, '\n')

# Evaluate model
mse, r2, adj = pipe.evaluate_model(grid_model, test_features, test_target)

d = {'model': grid_model_result,
     'mse': mse,
     'r2': r2}

best_models['lasso'] = d

# Rank coefficients
pipe.rank_coefs(grid_model_result, train_features.columns)

### Linear Regression

In [ ]:
pipeline_l = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)), ('linreg', LinearRegression())
])

params_l = {
          'poly__degree': (1,2)
         }

grid_model, grid_model_result, cv_results = pipe.run_gridsearch(pipeline, params, train_features, train_target, verbose=1)

cv_results

In [ ]:
print(grid_model_result.best_estimator_, '\n')

# Evaluate model
mse, r2, adj = pipe.evaluate_model(grid_model, test_features, test_target)

d = {'model': grid_model_result,
     'mse': mse,
     'r2': r2}

best_models['linear'] = d

# Rank coefficients
pipe.rank_coefs(grid_model_result, train_features.columns)

### Elastic Net

In [ ]:
pipeline_l = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)), ('en',ElasticNet())
])

params_l = {'en__alpha': alpha_range,
          'poly__degree': (1,2)
         }

grid_model, grid_model_result, cv_results = pipe.run_gridsearch(pipeline, params, train_features, train_target, verbose=1)

cv_results

In [ ]:
print(grid_model_result.best_estimator_, '\n')

# Evaluate model
pipe.evaluate_model(grid_model, test_features, test_target)

mse, r2, adj = pipe.evaluate_model(grid_model, test_features, test_target)

d = {'model': grid_model_result,
     'mse': mse,
     'r2': r2}

best_models['en'] = d

# Rank coefficients
pipe.rank_coefs(grid_model_result, train_features.columns)

In [ ]:
best_models